In [1]:
# Obejective: Predict whether the patient will get pregnant based on IVF treatment factors

In [2]:
#Importing Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#  load the dataset
df = pd.read_csv("HealthCareProject.csv")

In [4]:
#About the Dataset
df.head()

# v1 - Age
# v2 - Classifier - Primary, secondary i.e., sterility. >> 
#     (Primary infertility (The patient has never conceived before).Secondary infertility (The patient has conceived before but is now struggling with infertility))
# v3 - Years before pregnancy(Years after marriage) 
# v4 - Classifier - infertity by m, f, both m and f
# v5 - Classifier - whose egg we are using, self or donatd
# v6 - Classifier - male sperm (self or donor or combination of both)
# v7 - classifier - method to activate sperms, stimulation
# v8--  how we got sperms 4 to 5 methods, example masturbation, etc. --- outcome classifier.
# v9 - Categorical number - No of eggs.
# v10 - Predictive Outcomes - classifier - no of eggs formed - Intermediate outcomes - How many eggs formed 
# v11 - Predictive outcomes - classifier - How many eggs got converted to embryosss
# v12 - Does it come under input variable in first tier or second tier. Immediately or after frozen
# v13 – Pregnant (Yes/No)


,Unnamed: 0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13
0,METype,1,1,1.0,1,1,1,1,1,1,2,2,1,3
1,C1,32,2,3.5,3,1,1,2,1,5,2,3,2,1
2,C2,42,1,3.5,3,1,1,1,2,3,2,3,2,3
3,C3,29,2,3.5,2,1,1,1,2,8,7,8,2,1
4,C4,23,1,3.0,3,1,1,1,2,15,9,9,2,1


In [5]:
#dropped unnecessary  column and row and resetting index 

df = df.drop(columns='Unnamed: 0').drop(index=0).reset_index(drop=True)


In [6]:
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13
0,32,2,3.5,3,1,1,2,1,5,2,3,2,1
1,42,1,3.5,3,1,1,1,2,3,2,3,2,3
2,29,2,3.5,2,1,1,1,2,8,7,8,2,1
3,23,1,3.0,3,1,1,1,2,15,9,9,2,1
4,22,1,5.0,3,1,1,1,1,5,3,3,2,3


In [7]:
#checking for null values
df.isnull().sum()

v1     0
v2     0
v3     0
v4     0
v5     0
v6     0
v7     0
v8     0
v9     0
v10    0
v11    0
v12    0
v13    0
dtype: int64

In [8]:
#statistical View of columns (count, mean , STD, min ,max etc)
df.describe()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.00000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,31.482143,10.254464,20.089732,16.098214,23.491071,23.424107,210.433036,108.71875,122.812500,130.580357,130.723214,130.977679,2.348214
std,5.305860,94.058957,114.387135,114.776090,147.729573,147.739783,407.260689,309.09484,318.236608,328.984281,328.929578,335.493672,0.920224
min,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,28.000000,1.000000,3.375000,3.000000,1.000000,1.000000,1.000000,1.00000,5.000000,4.000000,4.000000,2.000000,1.000000
50%,31.000000,1.000000,6.000000,3.000000,1.000000,1.000000,1.000000,2.00000,8.000000,7.000000,7.000000,2.000000,3.000000
75%,35.000000,2.000000,10.000000,3.000000,1.000000,1.000000,1.000000,3.00000,12.000000,11.000000,10.000000,2.000000,3.000000
max,49.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.00000,999.000000,999.000000,999.000000,999.000000,3.000000


Noticed extreme values will replace it with forwardfill, we are going to use mask which changes extreme value with NaN, since NaN comes under float our dataset might turn in to float .. not that it changes anything but i just like int

In [9]:
for col in df.columns:
    df[col] = df[col].mask(df[col] == 999).ffill().astype(int)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   v1      224 non-null    int64
 1   v2      224 non-null    int64
 2   v3      224 non-null    int64
 3   v4      224 non-null    int64
 4   v5      224 non-null    int64
 5   v6      224 non-null    int64
 6   v7      224 non-null    int64
 7   v8      224 non-null    int64
 8   v9      224 non-null    int64
 9   v10     224 non-null    int64
 10  v11     224 non-null    int64
 11  v12     224 non-null    int64
 12  v13     224 non-null    int64
dtypes: int64(13)
memory usage: 22.9 KB


In [11]:
#checking for unique values in dataset having a look at data
# for col in df.columns:
#     unique_vals = df[col].unique()
#     num_unique = len(unique_vals)
#     print(f"Column: {col}")
#     print(f" - Unique values ({num_unique}): {unique_vals}\n")

#     print("*"*50)

In [12]:
# filtered_row = df.loc[df['v13'] == 1]
# print(filtered_row)

Determined that 1 is success and 2,3 failure ..hence mapping

In [13]:
df['v13'] = df['v13'].map({1: 1, 2: 0, 3: 0})

In [14]:
# Select all columns except the last one as features (X)
x = df.iloc[:,:-1]


# Select the last column ('v13') as the target variable (y)
y = df['v13']  


In [15]:
y.value_counts()

v13
0    155
1     69
Name: count, dtype: int64

to avoid model getting biased lets resample

In [16]:
from imblearn.over_sampling import RandomOverSampler

In [17]:
ros = RandomOverSampler()
x, y =ros.fit_resample(x,y)

In [18]:
x.shape

(310, 12)

In [19]:
y.shape

(310,)

In [20]:
y.value_counts()

v13
1    155
0    155
Name: count, dtype: int64

lets scale the values .. to bring it on same scale normal for tree model doesn't need scaling but to test out other model eventually we need scaling so why not do it now 

train test and split the model 

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20)

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier
Model_1 = RandomForestClassifier()

Model_1.fit(x_train, y_train)

Model_1.score(x_test, y_test)*100, Model_1.score(x_train, y_train)*100

(79.03225806451613, 97.17741935483872)

feature selection using forward elimanation 

In [25]:
# ### 1️⃣ Correlation Analysis ###
# plt.figure(figsize=(10,6))
# sns.heatmap(x.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
# plt.title("Feature Correlation Heatmap")
# plt.show()

randomforest initialy with default values

Using gridSearchCV to find out best parameter aka hyperparameter tuning

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = {
    "n_estimators": [5,10,15,20,25,30,35,100,200],  
    "criterion": ["gini", "entropy"],     
    "max_depth": [10,15,20,25,30,40,45]                
}

In [28]:
gdr = GridSearchCV(Model_1, param_grid=param_grid, cv=5)
gdr.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 15, 20, 25, 30, 40, 45],
                         'n_estimators': [5, 10, 15, 20, 25, 30, 35, 100, 200]})

In [29]:
gdr.best_params_  

{'criterion': 'gini', 'max_depth': 15, 'n_estimators': 30}

In [30]:
gdr.best_score_

np.float64(0.8067755102040817)

In [31]:
# hypertuneRf= RandomForestClassifier(criterion= 'entropy', max_depth=10, n_estimators= 150, random_state= 20)

In [32]:
# hypertuneRf.fit(x_train,y_train)
# hypertuneRf.score(x_test, y_test)* 100, hypertuneRf.score(x_train, y_train)*100

below suport vector  and decision tree classifer didn't provided best result hence commtting out 

In [33]:
# from sklearn.svm import SVC

In [34]:
# svc_model = SVC(kernel='poly')
# svc_model.fit(x_train,y_train)

In [35]:
# svc_model.score(x_test, y_test), svc_model.score(x_train,y_train)
# # (0.6451612903225806, 0.7862903225806451)

In [36]:
# psvc = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']}


In [37]:
# gdsv = GridSearchCV(svc_model, param_grid= psvc, cv=5)
# gdsv.fit(x_train, y_train)

In [38]:
# gdsv.best_params_

In [39]:
# gdsv.best_score_
#np.float64(0.6092244897959184)

In [40]:
# from sklearn.tree import DecisionTreeClassifier

In [41]:
# #'criterion': 'entropy',
#  'max_depth': 20,
#  'max_features': 'sqrt',
#  'random_state': 10}
# #
# #

In [42]:
# tree_model = DecisionTreeClassifier(criterion='entropy', max_depth= 20 , max_features= "sqrt", random_state= 10)
# tree_model.fit(x_train, y_train)

In [43]:
# #parameters
# pdstc= {"criterion":['gini', 'entropy', 'log_loss'],
# "max_depth": [10,20,30,40],
# "max_features": ['auto', 'sqrt', 'log2'] ,
# "random_state": [10,15,20,30,42]}

In [44]:
# gddtc = GridSearchCV(tree_model, param_grid= pdstc, cv=5)

# gddtc.fit(x_train, y_train)

In [45]:
# gddtc.best_params_

In [46]:
# tree_model.score(x_test,y_test), tree_model.score(x_train,y_train)

In [62]:
# from gridserach {'criterion': 'gini', 'max_depth': 15, 'n_estimators': 30}
# Train the Random Forest Model
model_2 = RandomForestClassifier(criterion='gini', max_depth=33 ,n_estimators= 10 )

model_2.fit(x_train, y_train)

model_2.score(x_test, y_test)*100, model_2.score(x_train, y_train)*100



(87.09677419354838, 97.17741935483872)

In [63]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [64]:
y_pred = model_2.predict(x_test)

results = pd.DataFrame({'Acutal': y_test.values, 'Predicted': y_pred})
print(results)



    Acutal  Predicted
0        0          1
1        0          1
2        0          0
3        1          1
4        1          1
..     ...        ...
57       0          0
58       1          0
59       1          1
60       0          0
61       0          0

[62 rows x 2 columns]


In [65]:
accuarcy = accuracy_score(y_test, y_pred)
print(f"Accuary : {accuarcy}")


Accuary : 0.8709677419354839


In [66]:
conf_mat = confusion_matrix(y_test,y_pred)
print(f"Confusion Matrix : \n {conf_mat}")

Confusion Matrix : 
 [[30  3]
 [ 5 24]]


In [52]:
cls_rep = classification_report(y_test, y_pred)
print(f"Classification Reort : \n {cls_rep}")

Classification Reort : 
               precision    recall  f1-score   support

           0       0.83      0.73      0.77        33
           1       0.73      0.83      0.77        29

    accuracy                           0.77        62
   macro avg       0.78      0.78      0.77        62
weighted avg       0.78      0.77      0.77        62

